### ACH RETURN QUERIES

#### ACH level fields

In [ ]:
"""
WITH 
  ach_transfer_info AS 
  (SELECT 
  iat.id AS ach_id, iat.bank_account_id, iat.cents, iat.requested AS ach_requested, iat.current_status, iat.user_id, iat.exchange_account_id, 
  RANK() OVER (PARTITION BY user_id ORDER BY requested DESC) as nth_transfer
  ,sare.result_id, sare.tran_code, sare.amount_in_cents, sare.direction, sare.effective_date AS ach_returned, sare.return_code
  ,sba.is_corporate, sba.created_at AS created_bankacc, sba.is_deleted, sba.is_primary, sba.ach_info_id, sba.account_type
  FROM incoming_ach_transfers AS iat
  LEFT JOIN scrubbed_ach_return_entries AS sare USING(id)
  LEFT JOIN scrubbed_bank_accounts AS sba ON iat.bank_account_id = sba.id
  WHERE iat.id IS NOT NULL)

, limits_at_ach AS 
  (SELECT 
  ach.ach_id, ach.exchange_account_id,
  coalesce(max(blc.daily_deposit_in_cents)/100,500) AS dailydollalimit, max(blc.thirty_day_deposit_in_cents)/1000 AS modollalimit, 
  coalesce(max(blc.completed_at),max(ach.created_bankacc)) AS limit_change_completed_at
  FROM ach_transfer_info AS ach
  LEFT JOIN bank_limit_changes AS blc ON (ach.exchange_account_id = blc.exchange_account_id) AND (ach.ach_requested >= blc.completed_at)
  GROUP BY ach.ach_id, ach.exchange_account_id)
  
, session_ach AS 
  (SELECT 
  session_start_time.*, us2.last_password_entry, us2.last_authy_entry, us2.last_price, us2.operating_system, us2.browser,
  us2.browser_version, us2.one_touch_enabled, us2.expired, us2.last_web_authn_entry
  
  FROM 
    (SELECT ach.ach_id, ach.exchange_account_id, ach.user_id, max(us.created) AS session_start
    FROM ach_transfer_info AS ach
    LEFT JOIN user_sessions AS us ON (ach.user_id = us.exchange_user_id) AND (ach.ach_requested >= us.created)
    GROUP BY ach.ach_id, ach.exchange_account_id, ach.user_id) AS session_start_time
    LEFT JOIN user_sessions AS us2 ON (session_start_time.user_id = us2.exchange_user_id) AND (session_start_time.session_start = us2.created))
  
SELECT 
transfer.*, 
limits.dailydollalimit, limits.modollalimit, limits.limit_change_completed_at,
sesh.session_start, sesh.last_password_entry, sesh.last_authy_entry, sesh.last_price, sesh.operating_system, sesh.browser, sesh.browser_version, sesh.one_touch_enabled, sesh.expired, sesh.last_web_authn_entry
FROM ach_transfer_info AS transfer
LEFT JOIN limits_at_ach AS limits ON transfer.ach_id = limits.ach_id
LEFT JOIN session_ach AS sesh ON transfer.ach_id = sesh.ach_id;
"""

#### User level fields

In [ ]:
"""
WITH
    account_info AS
    (SELECT 
    eu.id as user_id, uagr.account_group_id AS group_id, ea.id AS account_id,
    eu.tier, eu.first_verified_at, eu.created_at, eu.state_code, eu.country_code, sadm.is_institutional, sui.birthdate

    FROM scrubbed_exchange_users AS eu
    LEFT JOIN user_account_group_roles AS uagr ON (eu.id = uagr.exchange_user_id)
    LEFT JOIN user_account_roles AS uar ON (eu.id = uar.exchange_user_id)
    INNER JOIN scrubbed_exchange_accounts AS ea ON (ea.account_group_id = uagr.account_group_id OR ea.id = uar.exchange_account_id)
    LEFT JOIN scrubbed_account_derived_metadata AS sadm ON (ea.id = sadm.exchange_account_id)
    LEFT JOIN scrubbed_user_info AS sui ON (sui.exchange_user_id = uagr.exchange_user_id)
    WHERE eu.tier = 'Full')
    
, onfido_info AS 
  (SELECT  
  oa.exchange_user_id, 
  oc.onfido_applicant_id, oc.onfido_created_at, oc.gemini_created_at, oc.completed_at,
  odr.onfido_check_id, odr.result AS odr_result, odr.sub_result AS odr_subresult,
  sodp.onfido_report_id, sodp.date_of_birth, sodp.date_of_expiry, sodp.nationality, sodp.document_type, sodp.issuing_country
  
  FROM onfido_applicants AS oa
  LEFT JOIN onfido_checks AS oc USING(onfido_applicant_id) -- duplicates on this join (multiple checks per applicant)
  LEFT JOIN onfido_document_reports AS odr USING(onfido_check_id)
  LEFT JOIN scrubbed_onfido_document_properties AS sodp USING(onfido_report_id)
  WHERE oc.onfido_applicant_id IS NOT NULL
  ORDER BY exchange_user_id, onfido_check_id DESC)
  
SELECT *
FROM account_info AS acc
LEFT JOIN onfido_info AS onfido
ON acc.user_id = onfido.exchange_user_id;
"""

#### Market Conditions

In [ ]:
"""WITH 
  query1 AS
  (SELECT dcr.created, dcr.price, dcr2.created AS createdlag, dcr2.price AS pricelag,
  RANK() OVER (PARTITION BY dcr.created ORDER BY dcr2.created DESC) AS nthday
  
  FROM daily_conversion_rates AS dcr
  LEFT JOIN daily_conversion_rates AS dcr2 ON dcr.created::date >= dcr2.created::date AND dcr.created::date <= (dcr2.created::date + 4)
  WHERE dcr.trading_pair = 'BTCUSD' AND dcr2.trading_pair = 'BTCUSD' 
  ORDER by dcr.created, dcr2.created)

, query2 AS
  (SELECT * FROM query1
  WHERE created::date NOT IN (SELECT created::date FROM query1 GROUP BY created::date HAVING max(nthday) < 5 ORDER BY created ASC)
  ORDER BY created)

SELECT 
created::date, max(pricelag) - min(pricelag) as range,
(sum(pricelag * CAST(nthday = 5 as int)) - sum(pricelag * CAST(nthday = 1 as int)))/sum(pricelag * CAST(nthday = 1 as int)) as diffday4,
(sum(pricelag * CAST(nthday = 2 as int)) - sum(pricelag * CAST(nthday = 1 as int)))/sum(pricelag * CAST(nthday = 1 as int)) as diffday1
FROM query2
GROUP BY created::date
ORDER BY created;"""